# 1 Interpretation and Significance

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒=873+0.0012𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒+0.00002𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒2−223.57ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠

expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars, and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.

Need to ensure homoscedasticity in the error terms for the features. There may also be an issue of colinearity between the two annual incomes. We also need to ensure the errors are normally distributed and don't have a specific trend. Means would be good for the different incomes because that would help interpret why the coefficients are so small.

# 2. Weather Model

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sqlalchemy import create_engine
import warnings

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [6]:
import warnings
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor


def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")

<IPython.core.display.Javascript object>

In [3]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "weatherinszeged"
table_name = "weatherinszeged"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

df = pd.read_sql_query("select * from {}".format(table_name), con=engine)

# no need for an open connection,
# as we're only doing a single query
engine.dispose()

<IPython.core.display.Javascript object>

In [4]:
weather = df.copy()

<IPython.core.display.Javascript object>

In [5]:
weather.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


<IPython.core.display.Javascript object>

In [7]:
weather["temp_diff"] = weather["apparenttemperature"] - weather["temperature"]

<IPython.core.display.Javascript object>

In [33]:
X = weather[["humidity", "windspeed"]]
y = weather["temp_diff"]

<IPython.core.display.Javascript object>

In [34]:
X_const = sm.add_constant(X)

model = sm.OLS(y, X_const).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_diff   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        12:31:23   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

In [30]:
test_df = test_df.drop(columns=test_df.columns)

<IPython.core.display.Javascript object>

In [35]:
test_df = pd.concat([X, y], axis=1)
test_df.corr()
# I think I would expect the windspeed to negatively affect the apparent temp

,humidity,windspeed,temp_diff
humidity,1.000000,-0.224951,-0.242212
windspeed,-0.224951,1.000000,-0.411943
temp_diff,-0.242212,-0.411943,1.000000


<IPython.core.display.Javascript object>

In [32]:
test_df.head()

,humidity,windspeed,h/ww,ws/h,ws*h,temp_diff
0,0.89,14.1197,0.063033,15.864831,12.566533,-2.083333
1,0.86,14.2646,0.060289,16.586744,12.267556,-2.127778
2,0.89,3.9284,0.226555,4.413933,3.496276,0.000000
3,0.83,14.1036,0.058850,16.992289,11.705988,-2.344444
4,0.83,11.0446,0.075150,13.306747,9.167018,-1.777778


<IPython.core.display.Javascript object>

In [36]:
test_df["ws*h"] = test_df["windspeed"] * test_df["humidity"]

<IPython.core.display.Javascript object>

In [37]:
X = test_df.drop(columns="temp_diff")
y = test_df["temp_diff"]

<IPython.core.display.Javascript object>

In [38]:
X_const = sm.add_constant(X)

model = sm.OLS(y, X_const).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_diff   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        12:31:37   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0839      0.033      2.511      0.012       0.018       0.149
humidity       0.1775      0.043      4.133      0.000       0.093       0.262
windspeed      0.0905      0.002     36.797      0.000       0.086       0.095
ws*h          -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

The signs did change. The model is more accurate now as well. It is still showing statistical significance for all features but the original columns have less of an impact on the overall prediction.

# 3. House prices model

In [93]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"
table_name = "houseprices"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

df = pd.read_sql_query("select * from {}".format(table_name), con=engine)

# no need for an open connection,
# as we're only doing a single query
engine.dispose()

<IPython.core.display.Javascript object>

In [94]:
df = df.drop(columns=["poolqc", "miscfeature", "alley", "fence"])
df["fireplacequ"] = df["fireplacequ"].fillna("None")
df["lotfrontage"] = df["lotfrontage"].fillna(df["lotfrontage"].median())
df = df.drop(columns=["garagecars", "garagequal", "garagecond"])
df["garageyrblt"] = df["garageyrblt"].fillna(df["garageyrblt"].median())
df["garagefinish"] = df["garagefinish"].fillna("None")
df["garagetype"] = df["garagetype"].fillna("None")
bsmt_cols = ["bsmtfintype2", "bsmtexposure", "bsmtfintype1", "bsmtcond", "bsmtqual"]
for col in bsmt_cols:
    df[col] = df[col].fillna("None")
df["masvnrarea"] = df["masvnrarea"].fillna(0.0)
df["masvnrtype"] = df["masvnrtype"].fillna("None")
df = df.dropna()
df = df.drop(columns=["lotconfig", "utilities"])

dummy_df = df.copy()
# for col in df.columns:
#     if df[col].dtype == "object":
#         dummy_df = pd.concat(
#             [dummy_df, pd.get_dummies(df[col], drop_first=True)], axis=1
#         )
#         dummy_df = dummy_df.drop(columns=col)
dummy_df = pd.get_dummies(dummy_df, drop_first=True)

<IPython.core.display.Javascript object>

In [95]:
X = dummy_df.drop(columns=["id", "saleprice"])
y = dummy_df["saleprice"]

<IPython.core.display.Javascript object>

In [96]:
X_const = sm.add_constant(X)

results = sm.OLS(y, X_const).fit()

<IPython.core.display.Javascript object>

In [97]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.916
Method:                 Least Squares   F-statistic:                     72.33
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        12:58:43   Log-Likelihood:                -16600.
No. Observations:                1459   AIC:                         3.365e+04
Df Residuals:                    1234   BIC:                         3.484e+04
Df Model:                         224                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -8.194e+05   1.05e+06     -0.779      0.436   -2.88e+06    1.25e+06
mssubclass               -0.9465     81.574     -0.012      0.991    -160.986     159.093
lotfrontage              40.7739     41.959      0.972      0.331     -41.545     123.093
lotarea                   0.7543      0.109      6.890      0.000       0.540       0.969
overallqual            6943.9227   1016.423      6.832      0.000    4949.814    8938.031
overallcond            5959.6384    872.249      6.832      0.000    4248.383    7670.894
yearbuilt               335.6633     77.263      4.344      0.000     184.081     487.245
yearremodadd            100.0175     55.380      1.806      0.071      -8.633     208.668
masvnrarea               18.9795      5.776      3.286      0.001       7.649      30.310
bsmtfinsf1               17.1622      2.896      5.926      0.000      11.480      22.844
bsmtfinsf2                6.5395      5.998      1.090      0.276      -5.228      18.307
bsmtunfsf                -1.6734      2.776     -0.603      0.547      -7.119       3.772
totalbsmtsf              22.0282      4.070      5.412      0.000      14.043      30.014
firstflrsf               12.8223      6.222      2.061      0.040       0.615      25.029
secondflrsf              33.0497      5.452      6.062      0.000      22.353      43.747
lowqualfinsf            -10.9910     13.466     -0.816      0.415     -37.409      15.427
grlivarea                34.8806      5.481      6.364      0.000      24.127      45.634
bsmtfullbath            666.8708   1985.796      0.336      0.737   -3229.039    4562.780
bsmthalfbath           -985.7100   3033.854     -0.325      0.745   -6937.792    4966.372
fullbath               3132.6199   2208.427      1.418      0.156   -1200.067    7465.307
halfbath               1252.5421   2098.675      0.597      0.551   -2864.823    5369.907
bedroomabvgr          -3804.2996   1368.574     -2.780      0.006   -6489.289   -1119.310
kitchenabvgr          -1.398e+04   5621.863     -2.487      0.013    -2.5e+04   -2953.485
totrmsabvgrd           1191.7514    951.890      1.252      0.211    -675.750    3059.253
fireplaces             7162.5957   2560.430      2.797      0.005    2139.318    1.22e+04
garageyrblt              23.3912     58.896      0.397      0.691     -92.155     138.938
garagearea               28.0341      5.842      4.799      0.000      16.573      39.496
wooddecksf               12.8161      5.869      2.184      0.029       1.302      24.330
openporchsf               2.9579     11.540      0.256      0.798     -19.682      25.598
enclosedporch             5.1479     12.442      0.414      0.679     -19.262      29.558
threessnporch            32.3784     22.603      1.433      0.152     -11.966      76.722
screenporch            

<IPython.core.display.Javascript object>

In [63]:
def return_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    # print('VIF results\n-------------------------------')
    return pd.Series(vifs, index=x.columns)
    # print('-------------------------------\n')

<IPython.core.display.Javascript object>

In [98]:
vif_df = return_vif(X)

<IPython.core.display.Javascript object>

In [99]:
drop_cols = vif_df[vif_df.values > 5].index

<IPython.core.display.Javascript object>

In [100]:
drop_cols[1:]

Index(['mssubclass', 'overallqual', 'yearbuilt', 'bsmtfinsf1', 'bsmtfinsf2',
       'bsmtunfsf', 'totalbsmtsf', 'firstflrsf', 'secondflrsf', 'lowqualfinsf',
       ...
       'fireplacequ_TA', 'garagetype_Attchd', 'garagetype_Basment',
       'garagetype_BuiltIn', 'garagetype_Detchd', 'garagetype_None',
       'garagefinish_None', 'saletype_New', 'saletype_WD',
       'salecondition_Partial'],
      dtype='object', length=108)

<IPython.core.display.Javascript object>

In [47]:
# 'BrDale' ,'BrkSide' ,'Crawfor', 'IDOTRR', 'NPkVill', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'Timber', 'Veenker' ,'Feedr'
# [
#     "mssubclass",
#     "lotfrontage",
#     "bsmtfinsf2",
#     "bsmtunfsf",
#     "lowqualfinsf",
#     "bsmtfullbath",
#     "bsmthalfbath",
#     'fullbath',
#     'halfbath',
#     'totrmsabvgrd',
#     'garageyrblt',
#     'openporchsf',
#     'enclosedporch',
#     'threessnporch',
#     'miscval',
#     'yrsold',
#     'IR3',
#     'Reg',
#     'Blueste',
#     'BrDale',
# ]

['mssubclass',
 'lotfrontage',
 'bsmtfinsf2',
 'bsmtunfsf',
 'lowqualfinsf',
 'bsmtfullbath',
 'bsmthalfbath']

<IPython.core.display.Javascript object>

In [101]:
X = X.drop(columns=drop_cols[1:])

<IPython.core.display.Javascript object>

In [102]:
X_const = sm.add_constant(X)

results = sm.OLS(y, X_const).fit()

<IPython.core.display.Javascript object>

In [103]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     35.94
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        12:59:16   Log-Likelihood:                -17471.
No. Observations:                1459   AIC:                         3.519e+04
Df Residuals:                    1336   BIC:                         3.584e+04
Df Model:                         122                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  4.111e+05   1.71e+06      0.240      0.810   -2.95e+06    3.77e+06
lotfrontage             231.2970     62.216      3.718      0.000     109.245     353.349
lotarea                   1.2833      0.170      7.568      0.000       0.951       1.616
overallcond            5364.8148   1188.400      4.514      0.000    3033.482    7696.148
yearremodadd            276.6274     79.335      3.487      0.001     120.992     432.263
masvnrarea               72.2431      6.966     10.370      0.000      58.577      85.909
bsmtfullbath           8360.3141   2881.729      2.901      0.004    2707.108     1.4e+04
bsmthalfbath          -2832.9944   4889.205     -0.579      0.562   -1.24e+04    6758.361
fullbath               3.038e+04   2997.133     10.137      0.000    2.45e+04    3.63e+04
halfbath               1.752e+04   2473.405      7.083      0.000    1.27e+04    2.24e+04
bedroomabvgr           5452.5095   1751.846      3.112      0.002    2015.841    8889.178
kitchenabvgr          -2.711e+04   6034.007     -4.493      0.000    -3.9e+04   -1.53e+04
garagearea               93.0516      6.912     13.463      0.000      79.492     106.611
wooddecksf               42.9971      9.596      4.481      0.000      24.172      61.822
openporchsf              40.9306     18.333      2.233      0.026       4.967      76.895
enclosedporch            24.1156     19.598      1.230      0.219     -14.331      62.563
threessnporch            28.2740     38.134      0.741      0.459     -46.535     103.083
screenporch              95.7460     19.963      4.796      0.000      56.585     134.907
poolarea                 33.6610     28.592      1.177      0.239     -22.430      89.752
miscval                   0.8954      2.437      0.367      0.713      -3.885       5.676
mosold                  402.5128    408.609      0.985      0.325    -399.073    1204.098
yrsold                 -506.7901    852.536     -0.594      0.552   -2179.246    1165.665
mszoning_RH           -3079.7827   1.09e+04     -0.281      0.778   -2.45e+04    1.84e+04
street_Pave             8.91e+04   1.92e+04      4.630      0.000    5.14e+04    1.27e+05
lotshape_IR2           1983.0701   6996.136      0.283      0.777   -1.17e+04    1.57e+04
lotshape_IR3          -4.802e+04   1.44e+04     -3.333      0.001   -7.63e+04   -1.98e+04
lotshape_Reg          -2979.3903   2487.771     -1.198      0.231   -7859.753    1900.973
landcontour_HLS        1.209e+04   8454.820      1.430      0.153   -4497.104    2.87e+04
landcontour_Low       -1.776e+04   1.03e+04     -1.722      0.085    -3.8e+04    2474.246
landcontour_Lvl        6464.8852   6082.255      1.063      0.288   -5466.925    1.84e+04
landslope_Mod          1882.0830   6526.013      0.288      0.773   -1.09e+04    1.47e+04
landslope_Sev         -

<IPython.core.display.Javascript object>

In [104]:
vif_df = return_vif(X)

<IPython.core.display.Javascript object>

In [105]:
drop_cols = vif_df[vif_df.values > 5].index

<IPython.core.display.Javascript object>

In [106]:
drop_cols[1:]

Index([], dtype='object')

<IPython.core.display.Javascript object>